In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import boxcox
from statsmodels.tsa.seasonal import STL

In [ ]:
src_url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
df = pd.read_csv(src_url, index_col='Date')
df.index = pd.to_datetime(df.index)
df = df[df.index >= datetime.strptime('2010-01-01', '%Y-%m-%d')]

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
ax.plot(df['Wind'])

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))

ax.plot(boxcox(df['Wind'], lmbda=0.1))

In [ ]:
stl = STL(df['Wind'].resample('W').sum(), period=7, seasonal=365)

In [ ]:
res = stl.fit()

In [ ]:
fig = res.plot()

In [ ]:
plt.plot(df['Wind'].resample('W').sum() - res.seasonal)